In [17]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import Select
import pandas as pd
from time import sleep
from datetime import datetime
from datetime import timedelta
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
import json
import get_params as gp
import chk_new_res as cn
import build_payload as bp
import send2cal as cal
import logging
import re

# logging.basicConfig(filename='logs/booking.log', level=logging.INFO)

def check_new(df):
    if len(df.index) == 0:
        print("no booking")
#         logging.info('No new booking found.')
    else:
        print("new booking found")
#         logging.info('New booking found.')

def CountDaysMonth(Arrival,Departure):    
    from datetime import date, timedelta
    from dateutil.parser import parse
    from datetime import datetime
    BookedDaysPerMonth = {}
    d1 = parse(Arrival).date()
    d2 = parse(Departure).date()
    # this will give you a list containing all of the dates
    times = [d1 + timedelta(days=x) for x in range((d2-d1).days + 1)]
    # Creates a dictionary and increments if it exists 
    for t in times:
        month = t.strftime("%b")+'-'+t.strftime("%Y")
        if month in BookedDaysPerMonth:
            BookedDaysPerMonth[month] += 1
        else:
            BookedDaysPerMonth[month] = 1
    Dates_list=[]
    for k,v in BookedDaysPerMonth.items():
        Dates_list.append('{}-{}'.format(k,v))
    return Dates_list


sleep(15)

while True:
    start_time = datetime.now().strftime("%m/%d/%Y %H:%M:%S")
#     logging.info('Starting new booking check')
#     logging.info(start_time)

    cred = gp.get_params('login')
    browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#     browser = webdriver.Remote('http://bookings:4444/wd/hub', desired_capabilities=DesiredCapabilities.CHROME)
    browser.get(str(cred['url']))

    # Login 
    username=browser.find_element(by=By.XPATH, value="//input[@placeholder='Email']")
    password=browser.find_element(by=By.XPATH, value="//input[@placeholder='Password']")
    login=browser.find_element(by=By.XPATH, value="//button[@type='submit']")
    username.send_keys(str(cred['user']))
    password.send_keys(str(cred['password']))
    login.submit()

    sleep(5)

    ## click reservations tab
    restab=browser.find_element(by=By.XPATH, value="//span[normalize-space()='Reservations']")
    restab.click()
    sleep(5)

    ## Need to make sure that 100 is showing instead of 10
    select = Select(browser.find_element(by=By.XPATH, value="//*[@id='table_length']/label/select"))
    # select by value 
    select.select_by_value('100')

    # check if another page 
    table=browser.find_element(by=By.XPATH, value='//*[@id="table_paginate"]/ul')
    items = table.text.split()
    items.remove('Previous')
    items.remove('Next')

    ## download reservations 
    # export=browser.find_element(by=By.XPATH, value="//input[@value='Export']")
    # export.click()
    # sleep(2)
# multiple pages
    webtables = []
    print(items)
    for page in items:
        # set to 100 
        select = Select(browser.find_element(by=By.XPATH, value="//*[@id='table_length']/label/select"))
        # select by value 
        select.select_by_value('100')
        # get reservations from web
        html = browser.page_source
        table = pd.read_html(html)
        res_raw_df = table[0]
        webtables.append(res_raw_df)
        # click next page 
        nxt=browser.find_element(by=By.XPATH, value='//*[@id="table_next"]/a')
        nxt.click()  
    
    combined=pd.concat(webtables)
    res_df=combined[['Res. #', 'Status', 'Unit', 'Guest', 'Booked Date', 'Check-In','Checkout', 'Nights', 'Income']]


['1', '2']


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=103.0.5060.114)
Stacktrace:
#0 0x5613167dfcd3 <unknown>
#1 0x5613165e7968 <unknown>
#2 0x5613165ea7c7 <unknown>
#3 0x5613165ea66b <unknown>
#4 0x5613165ea92c <unknown>
#5 0x561316626456 <unknown>
#6 0x561316623dd8 <unknown>
#7 0x561316621864 <unknown>
#8 0x5613166204a5 <unknown>
#9 0x561316614189 <unknown>
#10 0x56131663cbb2 <unknown>
#11 0x561316613a43 <unknown>
#12 0x56131663ccbe <unknown>
#13 0x56131664fea8 <unknown>
#14 0x56131663caa3 <unknown>
#15 0x5613166123fa <unknown>
#16 0x561316613555 <unknown>
#17 0x5613168272bd <unknown>
#18 0x56131682b418 <unknown>
#19 0x56131681136e <unknown>
#20 0x56131682c078 <unknown>
#21 0x561316805bb0 <unknown>
#22 0x561316848d58 <unknown>
#23 0x561316848ed8 <unknown>
#24 0x561316862cfd <unknown>
#25 0x7fe5b96b4609 <unknown>


In [8]:
res_df

,Res. #,Status,Unit,Guest,Booked Date,Check-In,Checkout,Nights,Income
0,1096275,Checked Out,Legendary Times at Story Lake,Brenes,"Aug 24, 2022","Sep 23, 2022","Oct 2, 2022",9,$0.00
1,1096930,Checked Out,Legendary Times at Story Lake,Sarousi,"Aug 31, 2022","Oct 14, 2022","Oct 19, 2022",5,$562.27
2,1097362,Checked Out,Legendary Times at Story Lake,Cutcher,"Sep 5, 2022","Oct 6, 2022","Oct 12, 2022",6,$674.73
3,1098288,Confirmed,Legendary Times at Story Lake,Hool,"Sep 16, 2022","Feb 1, 2023","Feb 5, 2023",4,$468.18
4,1098795,Checked Out,Legendary Times at Story Lake,Reed,"Sep 22, 2022","Oct 22, 2022","Oct 30, 2022",8,$899.64
5,1099938,Confirmed,Legendary Times at Story Lake,rippy,"Oct 5, 2022","Dec 23, 2022","Jan 2, 2023",10,"$2,582.61"
6,1100775,Confirmed,Legendary Times at Story Lake,Russell,"Oct 14, 2022","Nov 23, 2022","Nov 26, 2022",3,$506.43
7,1102386,Confirmed,Legendary Times at Story Lake,Philbrook,"Nov 1, 2022","Nov 19, 2022","Nov 23, 2022",4,$421.51
8,1102644,Checked Out,Legendary Times at Story Lake,Figueroa,"Nov 3, 2022","Nov 6, 2022","Nov 9, 2022",3,$314.41
9,1102691,Confirmed,Legendary Times at Story Lake,Moore,"Nov 4, 2022","Dec 3, 2022","Dec 10, 2022",7,$733.63


In [9]:
    res_file=gp.get_params('reservations')
    # check if res id exists
    curr = pd.read_csv(res_file['all_res'])
    new_res=res_df[['Res. #','Guest','Check-In','Checkout','Nights']]
    new_res.columns = ['Res_ID','Guest','CheckIn','CheckOut','Nights']
    n_df=cn.checkNewRes(curr,new_res)

    # Check if new
    check_new(n_df)

no booking


In [ ]:
    # Send new to calendar
    logging.info("Updating Calendar")
    result = n_df.to_json(orient="records")
    n_pl=json.loads(result)
    for rec in n_pl:
        cal.Send2Cal(bp.build_pl(rec))

    # save updated file with new booking
    update =[n_df,curr]
    master_df = pd.concat(update)
    master_df.reset_index(drop=True,inplace=True)
    master_df.to_csv(res_file['all_res'],index=False)
    logging.info("Updating Master List")

    # create a summary table 
    logging.info("Creating Summary Table")

In [22]:
    res_df=res_df.copy()
    res_df["Booked Date"] = pd.to_datetime(res_df["Booked Date"]).dt.strftime('%b/%d/%Y')
    res_df["Check-In"] = pd.to_datetime(res_df["Check-In"]).dt.strftime('%b/%d/%Y')
    res_df["Checkout"] = pd.to_datetime(res_df["Checkout"]).dt.strftime('%b/%d/%Y')
    # merge with old data
    rpts=gp.get_params('reports')
    old=pd.read_csv(rpts['old'])
    frames = [res_df,old]
    summ=pd.concat(frames)

In [23]:
summ

,Res. #,Status,Unit,Guest,Booked Date,Check-In,Checkout,Nights,Income
0,1096275,Checked Out,Legendary Times at Story Lake,Brenes,Aug/24/2022,Sep/23/2022,Oct/02/2022,9,$0.00
1,1096930,Checked Out,Legendary Times at Story Lake,Sarousi,Aug/31/2022,Oct/14/2022,Oct/19/2022,5,$562.27
2,1097362,Checked Out,Legendary Times at Story Lake,Cutcher,Sep/05/2022,Oct/06/2022,Oct/12/2022,6,$674.73
3,1098288,Confirmed,Legendary Times at Story Lake,Hool,Sep/16/2022,Feb/01/2023,Feb/05/2023,4,$468.18
4,1098795,Checked Out,Legendary Times at Story Lake,Reed,Sep/22/2022,Oct/22/2022,Oct/30/2022,8,$899.64
5,1099938,Confirmed,Legendary Times at Story Lake,rippy,Oct/05/2022,Dec/23/2022,Jan/02/2023,10,"$2,582.61"
6,1100775,Confirmed,Legendary Times at Story Lake,Russell,Oct/14/2022,Nov/23/2022,Nov/26/2022,3,$506.43
7,1102386,Confirmed,Legendary Times at Story Lake,Philbrook,Nov/01/2022,Nov/19/2022,Nov/23/2022,4,$421.51
8,1102644,Checked Out,Legendary Times at Story Lake,Figueroa,Nov/03/2022,Nov/06/2022,Nov/09/2022,3,$314.41
9,1102691,Confirmed,Legendary Times at Story Lake,Moore,Nov/04/2022,Dec/03/2022,Dec/10/2022,7,$733.63


In [19]:
pd.to_datetime(res_df["Booked Date"]).dt.strftime('%b/%d/%Y')

0     Aug/24/2022
1     Aug/31/2022
2     Sep/05/2022
3     Sep/16/2022
4     Sep/22/2022
5     Oct/05/2022
6     Oct/14/2022
7     Nov/01/2022
8     Nov/03/2022
9     Nov/04/2022
0     Aug/24/2022
1     Aug/31/2022
2     Sep/05/2022
3     Sep/16/2022
4     Sep/22/2022
5     Oct/05/2022
6     Oct/14/2022
7     Nov/01/2022
8     Nov/03/2022
9     Nov/04/2022
10    Nov/07/2022
11    Nov/09/2022
Name: Booked Date, dtype: object

In [24]:
res_df=res_df.copy()
res_df["Booked Date"] = pd.to_datetime(res_df["Booked Date"]).dt.strftime('%b/%d/%Y')
res_df["Check-In"] = pd.to_datetime(res_df["Check-In"]).dt.strftime('%b/%d/%Y')
res_df["Checkout"] = pd.to_datetime(res_df["Checkout"]).dt.strftime('%b/%d/%Y')

In [21]:
res_df

,Res. #,Status,Unit,Guest,Booked Date,Check-In,Checkout,Nights,Income
0,1096275,Checked Out,Legendary Times at Story Lake,Brenes,Aug/24/2022,Sep/23/2022,Oct/02/2022,9,$0.00
1,1096930,Checked Out,Legendary Times at Story Lake,Sarousi,Aug/31/2022,Oct/14/2022,Oct/19/2022,5,$562.27
2,1097362,Checked Out,Legendary Times at Story Lake,Cutcher,Sep/05/2022,Oct/06/2022,Oct/12/2022,6,$674.73
3,1098288,Confirmed,Legendary Times at Story Lake,Hool,Sep/16/2022,Feb/01/2023,Feb/05/2023,4,$468.18
4,1098795,Checked Out,Legendary Times at Story Lake,Reed,Sep/22/2022,Oct/22/2022,Oct/30/2022,8,$899.64
5,1099938,Confirmed,Legendary Times at Story Lake,rippy,Oct/05/2022,Dec/23/2022,Jan/02/2023,10,"$2,582.61"
6,1100775,Confirmed,Legendary Times at Story Lake,Russell,Oct/14/2022,Nov/23/2022,Nov/26/2022,3,$506.43
7,1102386,Confirmed,Legendary Times at Story Lake,Philbrook,Nov/01/2022,Nov/19/2022,Nov/23/2022,4,$421.51
8,1102644,Checked Out,Legendary Times at Story Lake,Figueroa,Nov/03/2022,Nov/06/2022,Nov/09/2022,3,$314.41
9,1102691,Confirmed,Legendary Times at Story Lake,Moore,Nov/04/2022,Dec/03/2022,Dec/10/2022,7,$733.63


In [30]:
    summ1=summ[['Check-In','Checkout', 'Nights', 'Income']]
    nightly_rate = []
    for i,b in summ1.iterrows():
        Arrival = b['Check-In']
        Departure = b['Checkout']
        Income_raw = b['Income']
        Income= float(re.sub(r'[^\d.]', '', Income_raw))
        times=CountDaysMonth(Arrival,Departure)

        times_df = pd.DataFrame(times) 
        times_df.to_csv(rpts['test'],index=False)
        summ1.to_csv(rpts['test2'],index=False)

        for t in times:
            nightly = Income / len(times)
            night_form = '{0:.2f}'.format(nightly)
            t_stmp=pd.Timestamp(t)
            nightly_rate.append([t_stmp,float(night_form)])
    breakdown = pd.DataFrame(nightly_rate, columns = ['Date', 'Per Night'])
    breakdown['Month-Year']=breakdown['Date'].dt.strftime('%m/%Y')
    rpts=gp.get_params('reports')
    breakdown.to_csv(rpts['breakdown'],index=False)

In [36]:
nightly_rate

[[Timestamp('2022-09-08 00:00:00'), 0.0],
 [Timestamp('2022-10-02 00:00:00'), 0.0],
 [Timestamp('2022-10-06 00:00:00'), 562.27],
 [Timestamp('2022-10-07 00:00:00'), 674.73],
 [Timestamp('2023-02-05 00:00:00'), 468.18],
 [Timestamp('2022-10-09 00:00:00'), 899.64],
 [Timestamp('2022-12-09 00:00:00'), 1291.31],
 [Timestamp('2023-01-02 00:00:00'), 1291.31],
 [Timestamp('2022-11-04 00:00:00'), 506.43],
 [Timestamp('2022-11-05 00:00:00'), 421.51],
 [Timestamp('2022-11-04 00:00:00'), 314.41],
 [Timestamp('2022-12-08 00:00:00'), 733.63],
 [Timestamp('2022-09-08 00:00:00'), 0.0],
 [Timestamp('2022-10-02 00:00:00'), 0.0],
 [Timestamp('2022-10-06 00:00:00'), 562.27],
 [Timestamp('2022-10-07 00:00:00'), 674.73],
 [Timestamp('2023-02-05 00:00:00'), 468.18],
 [Timestamp('2022-10-09 00:00:00'), 899.64],
 [Timestamp('2022-12-09 00:00:00'), 1291.31],
 [Timestamp('2023-01-02 00:00:00'), 1291.31],
 [Timestamp('2022-11-04 00:00:00'), 506.43],
 [Timestamp('2022-11-05 00:00:00'), 421.51],
 [Timestamp('2022-

In [29]:
def CountDaysMonth(Arrival,Departure):    
    from datetime import date, timedelta
    from dateutil.parser import parse
    from datetime import datetime
    BookedDaysPerMonth = {}
    d1 = parse(Arrival).date()
    d2 = parse(Departure).date()
    # this will give you a list containing all of the dates
    times = [d1 + timedelta(days=x) for x in range((d2-d1).days + 1)]
    # Creates a dictionary and increments if it exists 
    for t in times:
        month = t.strftime("%b")+'-'+t.strftime("%Y")
        if month in BookedDaysPerMonth:
            BookedDaysPerMonth[month] += 1
        else:
            BookedDaysPerMonth[month] = 1
    Dates_list=[]
    for k,v in BookedDaysPerMonth.items():
        Dates_list.append('{}-{}'.format(k,v))
    return Dates_list